## 딥러닝 프레임워큰 동작방식 

1. '정적 계산 그래프' : Define-and-Run  
2. '동적 계산 그래프' : Define-by-Run 

# Define-and-Run(정적 계산 그래프 방식)

Define-and-Run 직역 : '계산 그래프를 정의한 다음 데이터를 흘려보낸다'  
계산 그래프 정의는 사용자가 제공  
프레임워크는 주어진 그래프를 검퓨터가 처리할 수 있는 형태로 변환하여 데이터를 흘려보내는 식

컴파일에 의해 계산 그래프가 메모리상에 펼쳐지며, 실제 데이터를 흘려보낼 준비가 갖춰진다.  
**'계산 그래프 정의'와 '데이터 흘려보내기' 처리가 분리되어있음.**

수도코드
~~~python
# 가상의 Define-and-Run 방식 프레임워크용 코드 예

# 계산 그래프 정의 
a = Variable('a')
b = Variable('b')
c = a + b 
d = c + Constant(1)

# 계산 그래프 컴파일
f = compile(d)

# 데이터 흘려보내기 
d = f(a=np.array(2), b=np.array(3))
~~~

계산 그래프 정의 부분 4 줄의 코드에서 실제 계산이 이루어지지 않는다.  
실제 '수치'가 아닌 '기호'를 대상으로 프로그래밍 됐다. 이런 프로그래밍 방식을 '기호 프로그래밍'이라고 한다.  

Define-and-Run 방식 프레임워크에서는 실제 데이터가 아닌 기호를 가용한 추상적인 계산 절차를 코딩해야 한다.  
그리고 도메인 특화 언어(DSL)를 사용해야 한다.  
(도메인 특화 언어 : 프레임워크 자체의 규칙들로 이루어진 언어)


**NOTE_**
Define-and-Run 방식 프레임워크의 대부분은 도메인 특화언어를 사용하여 계산을 정의한다.  
도메인 특화언어는 한마디로 '파이썬 위에서 동작하는 새로운 프로그래밍 언어'라고 할수 있다.  
그리고 미분을 하기위해 설계된 언어이다. --> **'미분 가능 프로그래밍'**(differentiable programming)

딥러닝 여명기에 Define-and-Run 방식프레임워크가 대부분  
텐서플로우(1.x), 카페, CNTK


## Define-and-Run(정적 계산 그래프 방식)의 단점

정적 계산 그래프 프레임워크에서는 컴파일을 거쳐 프레임워크만 이해하고 실행할 수 있는 표현 형식으로 변환  
파이썬은 이 독자적인 표현 형식을 이해할 수 없다.

디버깅어 어려운 본질적인 이유 : '계산 그래프 정의'와 '데이터 흘려보내기' 작업이 분리되어 있다.  
문제(버그)는 주로 '데이터 흘려보낼때' 발견되지만, 문제 원인은 '계산 그래프 정의'에 있는 경우가 대부분이다.  
--> 문제 발생 시점과 원인이 만들어지는 시점이 떨어져 있어서 어디가 문제인지 특정하기 어려울 때가 많다.

**NOTE_**
정적 계산 그래프(Define-and-Run) 프레임워크는 데이터를 흘려보내기에 앞서 계산 그래프를 정의해야한다.  
따라서 데이터를 흘랴버내는 동안은 계산 그래프의 구조를 바꿀 수 없다.  
또한 if문을 사용못하고 그에 대응하는 tf.cond 같은 전용 연산의 사용법을 익혀야 해서 프로그래머가 도메인 특화 언어를 새롭게 배워야하는 부담이 생긴다.

## Define-and-Run(정적 계산 그래프 방식)의 장점

1. 정적 계산 그래프는 성능이 높다.  
계산 그래프를 최적화하면 성능도 따라서 최적화된다.  
그래서 계산 그래프 최적화는 계산 그래프의 구조와 사용되는 연산을 효율적인 것으로 변환하는 형태로 이뤄진다.  
두 개의 연산을 하나의 연산으로 축약하여 계산 시간이 단축

Define-and-Run 방식의 프레임워크는 데이터를 흘려보내기 전에 전체 계산 그래프가 손에 들어오므로 계산 그래프 전체를 고려해 최적화할 수 있다. 예를 들어 for문 등에서 반복해 사용하는 연산을 하나로 '축약'하여 계산 효율을 크게 끌어 올릴 수 있는 경우도 있다.

2. 어떻게 컴파일 하느냐에 따라 다른 실행 파일로 변환할 수도 있다.  
따라서 파이썬이 아닌 다른 환경에서도 데이터를 흘려보내는 게 가능하다.(파이썬에서 벗어났을 때 얻는 가장 큰 혜택은 파이썬 자체가 주는 오버헤드가 사라진다는 것)

3. 학습을 여러 대의 컴퓨터에 분산해 수행할때 유리하다.


# Define-by-Run(동적 계산 그래프 방식)

Define-by-Run 직역 : '데이터를 흘려보냄으로써 계산 그래프가 정의된다'  
'데이터 흘려보내기'와 '계산 그래프 구축'이 동시에 이루어진다.

사용자가 데이터를 흘려보낼 때(일반적인 수치 계산을 수행할 때) 자동으로 계산 그래프를 구성하는 '연결(참조)'를 만든다.  
구현 수준에서는 '연결 리스트'로 표현되는데, 연결 리스트를 사용하면 계산이 끝난 후 해당 연결을 역방향으로 추적할 수 있기 때문

Define-by-Run 방식 프레임워크는 넘파이를 사용하는 일반적인 프로그래밍과 똑같은 형태로 코딩할 수 있다.  

Define-by-Run 방식은 2015년에 체이너에 의해 처음 제창되고 이후 많은 프레임워크에 채용되고 있다.  

대표적으로 파이토치, MXNet, DyNet, 텐서플로우(2.x)

## Define-by-Run(동적 계산 그래프 방식)의 장점

1. 동적 계산 그래프 프레임워크에서는 일반 넘파이를 사용할 때외 같은 방식으로 수치계산이 가능  
따라서 프레임워크 고유의 '도메인 특화 언어'를 배우지 않아도 된다.  
계산 그래프를 '컴파일'하여 독자적인 데이터 구조로 변환할 필요도 없다. : 일반 파이썬 프로그래밍으로 계산 그래프를 구축하고 실행할 수 있다.  while문, for문, if문 등을 그대로 사용하여 계산 그래프를 만들수 있다.  
그러면 '계산 그래프' 또는 '계산 그래프를 이루는 연결'이 자동으로 만들어진다.  

2. 디버깅에 유리  
계산 그래프가 파이썬 프로그램 형태로 실행되기 때문에 디버깅도 항상 파이썬 프로그램으로 할 수 있다.


# 정리

|       | Define-and-Run(정적 계산 그래프) | Define-by-Run(동적 계산 그래프) |
| ----- | ------------------------------- | ------------------------------- |
| 장점  | - 성능이 좋다 <br/> - 신경망 구조를 최적화하기 쉽다. <br/> - 분산 학습시 더 편리하다. | - 파이썬으로 계산 그래프를 제어할 수 있다. <br/> - 디버깅이 쉽다. <br/> - 동적인 계산 처리에 알맞다. |
| 단점  | - 독자적인 언어(규칙)를 익혀야 한다. <br/> - 동적 계산 그래프를 만들기 어렵다. <br/> - 디버깅하기 매우 어려울 수 있다. | - 성능이 낮을 수 있다. | 
